# How to implement a positioning of objects

In [ ]:
# First install py2vision
!pip install py2vision

In [ ]:
# Import all necessary
import os

from py2vision.input_output.vision_system import VisionSystem
from py2vision.input_output.camera import Camera
from py2vision.stereo.standard_stereo import StandardStereo
from py2vision.stereo.match_method import Matcher, StereoSGBM
from py2vision.recognition.yolov3_detector import ObjectDetectorYoloV3
from py2vision.recognition.selector import Recognizer

## Relevant paths
**You can get coco.names here https://github.com/pjreddie/darknet/blob/master/data/coco.names**

In [ ]:
classes_file = "/content/coco.names"
work_dir = "/content/work_dir"
stereo_maps_path = "stereoMap"

## Create work directory

In [ ]:
try:
    os.mkdir(work_dir)
except:
    pass

## Calibrate and rectify cameras

In [ ]:
left_camera = Camera("left_camera", "LEFT_IMAGE")
right_camera = Camera("right_camera", "RIGHT_IMAGE")
stereo_pair_fisheye = StandardStereo(left_camera, right_camera)
stereo_pair_fisheye.calibrate("PUT_LEFT_CALIBRATION_IMAGE_FOLDER", "PUT_RIGHT_CALIBRATION_IMAGE_FOLDER", show=False)
stereo_pair_fisheye.rectify((640, 720), (640, 720), export_file=True, export_file_name=stereo_maps_path)

## Configure matcher

In [ ]:
# Add path format
stereo_maps_path = stereo_maps_path + ".xml"
sgbm = StereoSGBM(min_disp=-32, max_disp=32, window_size=3, p1=107, p2=710, pre_filter_cap=36, speckle_window_size=117, speckle_range=5, uniqueness_ratio=3, disp_12_max_diff=-38)
matcher = Matcher(sgbm)
lmbda = 13673
sigma = 1.3175

# Restore model weights
**You can get COCO weights here https://pjreddie.com/media/files/yolov3.weights

In [ ]:
yolov3 = ObjectDetectorYoloV3("test", 80, training=False)
recognizer = Recognizer(yolov3)
weights_file = "PLACE_WEIGHTS_PATH_HERE"
recognizer.restore_weights(weights_file)

## See your model architecture

In [ ]:
model = recognizer.get_model()

## Image positioning pipeline

In [ ]:
vis_sys = VisionSystem(left_camera, right_camera, stereo_maps_path, matcher, stereo_pair_fisheye.Q)
# Here the magic happens
vis_sys.image_pipeline(model, classes_file, os.path.join(work_dir, "test_position.jpg"),  lmbda=lmbda, sigma=sigma, downsample_for_match=None, show_window=True, score_threshold=0.5, iou_threshold=0.6, otsu_thresh_inverse=True, text_colors=(0, 0, 0))